# Combine product concentrations from GC with enzymatic assays and OD-pH-pressure

## Imports and setup

In [19]:
import os
from pathlib import Path
import webbrowser
from functools import reduce
import random
import operator
import pandas as pd
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import plotly.offline as py
import plotly.graph_objs as go
import plotly_express as px
import numpy as np
import sympy as sp
import qgrid as qg
import plotly.io as pio

from utils.plot_utils import scatterplot_from_df, boxplot_from_df 

idx = pd.IndexSlice

sp.init_printing()
py.init_notebook_mode(connected=True)
qg.set_defaults(show_toolbar=True)
qg.enable()

## Input files and definitions

In [23]:
means_CO2H2 = Path("./intermediate/mean_data_CO2H2.csv")
means_CO = Path("./intermediate/mean_data_CO.csv")

results_path = Path("./results/Comparison")

In [24]:
mean_data = pd.concat(
    [
        pd.read_csv(means_CO2H2).assign(Fermentation="CO2-H2"),
        pd.read_csv(means_CO).assign(Fermentation="CO")
    ],
    sort=False
).set_index(
    ["Fermentation", "Sample", "Condition", "Quality"]
).sort_index()

mean_data

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Some Defs for plotting

In [25]:
units_of_choice = {
    "Time": "h",
    "Concentration": "g/L",
    "Optical Extinction": "1",
    "pH": "1",
    "Absolute Pressure": "bar",
    "Relative Expression": "1",
    "Comparative Expression": "1"
}

quality_color_map = {
    "Acetate": "rgb(150,0,0)",
    "EtOH": "rgb(0,50,150)",
    "Fructose": "rgb(0,150,250)",
    "Formic Acid": "rgb(30,200,0)",
    "OD 600": "rgb(0,100,50)",
    "pH": "rgb(150,0,150)",
    "Pressure after": "rgb(0,0,50)",
    "aor": "rgb(250, 0, 150)",
    "pta": "rgb(200, 110, 0)",
    "fdh": "rgb(0, 150, 30)",
    "codh": "rgb(0, 30, 150)"
}

fermentation_color_map = {
    "CO": "rgb(200,0,0)",
    "CO2-H2": "rgb(0,75,200)"
}

In [29]:
def gen_plotly_layout(axis_map, measures):
    return {
        "xaxis": dict(
            domain=[0, (0.92 if len(measures) > 1 else 1)],
            title="Time"+" [%s]"%(units_of_choice["Time"]),
        ),
        **{
            "yaxis"+(str(ops["index"]) if ops["index"] != 1 else ""): dict(
                title=quant+" [%s]"%(units_of_choice[(quant)]),
                anchor="x",
                side=ops["position"],
                overlaying=("y" if ops["index"] != 1 else None)
            ) for quant, ops in axis_map.items()
        }
    }

def plot_measurements(df, measures, axis_map):
    def create_scatter(m, f, c, yaxis):
        data = df.reset_index(
        ).pipe(
            lambda df: df.loc[
                (df.Fermentation == f) &
                (df.Condition == c) &
                (df.Quality == m[0]) &
                (df.Quantity == m[1])
            ]
        )
        
        return go.Scatter(
            x=data.Time,
            y=data.Value,
            error_y=dict(
                type='data',
                array=data.StDev,
                visible=(True if data.StDev.sum() > 0 else False)
            ),
            mode="lines+markers",
            name="%s at %s"%(
                f,
                "5 atm" if c == "pressure" else "1 atm"
            ),
            marker=dict(
                color=fermentation_color_map[f]
            ),
            line=dict(
                dash=("dash" if c == "control" else None)
            ),
            opacity=0.8,
            yaxis=yaxis
        )
    
    scatters = [
        (
            create_scatter(
                m, 
                f, 
                c,
                "y"+str(axis_map[m[1]]["index"])
            )
        ) 
        for m, mode in measures.items() 
        for c in ["control", "pressure"] 
        for f in ["CO", "CO2-H2"]
    ]
    
    return dict(
        data=scatters,
        layout=gen_plotly_layout(axis_map, measures)
    )

## Pressure scatter

In [36]:
axis_map = {
    "Absolute Pressure": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(plot_measurements(
    mean_data,
    {
        ("Pressure after", "Absolute Pressure"): "mean"
    },
    axis_map
))

pio.write_image(fig, str(results_path / 'pressure.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0.01732051,        nan, 0.01258306, 0.02217356, 0.022…

## OD_600 scatter

In [35]:
axis_map = {
    "Optical Extinction": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        mean_data, 
        {
            ("OD 600", "Optical Extinction"): "mean"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'od-600.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0.04844241, 0.05042486, 0.02533114, 0.11100863, 0.016…

## Acetate scatter

In [37]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        mean_data, 
        {
            ("Acetate", "Concentration"): "mean"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'acetate.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0.05749706, 0.00079877, 0.17557194, 0.13070085, 0.140…

## Ethanol scatter

In [38]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        mean_data, 
        {
            ("EtOH", "Concentration"): "mean"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'ethanol.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([       nan,        nan, 0.29096017, 0.05587874, 0.073…

## Fructose

In [39]:
axis_map = {
    "Concentration": {
        "index": 1,
        "position": "left"
    }
}

fig = go.FigureWidget(
    plot_measurements(
        mean_data, 
        {
            ("Fructose", "Concentration"): "mean"
        }, 
        axis_map
    )
)

pio.write_image(fig, str(results_path / 'fructose.svg'))
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0.01100814, 0.01243729, 0.00296945, 0.00255934]),
   …